In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
root_path = '/content/drive/MyDrive/Colab Notebooks/Federated Learning/Final_code/GridSearch_Adam_result/'
# Định nghĩa các file JSON
eta_files = {
    0.1: root_path + "fl_results_gs_cl/FedAdam_LR_0_1.json",
    0.01: root_path +"fl_results_gs_cl/FedAdam_LR_0_01.json",
    0.001: root_path +"fl_results_gs_cl/FedAdam_LR_0_001.json",
    0.03162: root_path +"fl_results_gs_cl/FedAdam_LR_0_003162.json",
    0.003981: root_path +"fl_results_gs_cl/FedAdam_LR_0_003981.json"
}
# Load dữ liệu từ các file JSON
results = {}
for eta, file_path in eta_files.items():
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            results[eta] = data
        print(f"Đã load thành công file {file_path}")
    except FileNotFoundError:
        print(f"Không tìm thấy file {file_path}")
    except json.JSONDecodeError:
        print(f"Lỗi khi parse JSON file {file_path}")

# Tính toán các metrics
analysis_data = []
for eta in results.keys():
    metrics = results[eta]['metrics']

    # Final validation accuracy (trung bình 10 round cuối)
    val_accs = [round_data['val_acc'] for round_data in metrics]
    last_10_val_accs = val_accs[-10:]
    final_val_acc = np.mean(last_10_val_accs)

    # Best validation accuracy (cao nhất trong tất cả rounds)
    best_val_acc = max(val_accs)

    # Convergence stability (độ lệch chuẩn của validation accuracy trong 10 rounds cuối)
    convergence_stability = np.std(last_10_val_accs)

    analysis_data.append({
        'eta': eta,
        'final_val_acc': final_val_acc,
        'best_val_acc': best_val_acc,
        'convergence_stability': convergence_stability
    })

# Sắp xếp theo eta
analysis_data.sort(key=lambda x: x['eta'])

# Tìm eta tốt nhất dựa trên final validation accuracy
best_eta = max(analysis_data, key=lambda x: x['final_val_acc'])['eta']

# Tạo biểu đồ tương tự như trong hình
plt.style.use('default')
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

etas = [data['eta'] for data in analysis_data]
final_val_accs = [data['final_val_acc'] for data in analysis_data]
best_val_accs = [data['best_val_acc'] for data in analysis_data]
convergence_stabilities = [data['convergence_stability'] for data in analysis_data]

# 1. Final Validation Accuracy vs η (trung bình 10 round cuối)
ax1.plot(etas, final_val_accs, 'o-', color='blue', linewidth=2, markersize=8, label='Final Val Acc (Avg 10)')
ax1.axvline(x=best_eta, color='red', linestyle='--', linewidth=2, label=f'Best η={best_eta}')
ax1.set_title('Final Validation Accuracy vs η (Avg Last 10 Rounds)', fontsize=14, fontweight='bold')
ax1.set_xlabel('η (Learning Rate)', fontsize=12)
ax1.set_ylabel('Final Validation Accuracy (Avg)', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.legend()

# Thêm giá trị trên điểm
for i, (eta, acc) in enumerate(zip(etas, final_val_accs)):
    ax1.annotate(f'{acc:.3f}', (eta, acc), textcoords="offset points",
                xytext=(0,10), ha='center', fontsize=10)

# 2. Best Validation Accuracy vs η
ax2.plot(etas, best_val_accs, 'o-', color='green', linewidth=2, markersize=8, label='Best Val Acc')
ax2.axvline(x=best_eta, color='red', linestyle='--', linewidth=2, label=f'Best η={best_eta}')
ax2.set_title('Best Validation Accuracy vs η', fontsize=14, fontweight='bold')
ax2.set_xlabel('η (Learning Rate)', fontsize=12)
ax2.set_ylabel('Best Validation Accuracy', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.legend()

# Thêm giá trị trên điểm
for i, (eta, acc) in enumerate(zip(etas, best_val_accs)):
    ax2.annotate(f'{acc:.3f}', (eta, acc), textcoords="offset points",
                xytext=(0,10), ha='center', fontsize=10)

# 3. Convergence Stability vs η
ax3.plot(etas, convergence_stabilities, 'o-', color='red', linewidth=2, markersize=8, label='Convergence Stability')
ax3.axvline(x=best_eta, color='red', linestyle='--', linewidth=2, label=f'Best η={best_eta}')
ax3.set_title('Convergence Stability vs η', fontsize=14, fontweight='bold')
ax3.set_xlabel('η (Learning Rate)', fontsize=12)
ax3.set_ylabel('Convergence Stability (Std)', fontsize=12)
ax3.grid(True, alpha=0.3)
ax3.legend()

# Thêm giá trị trên điểm
for i, (eta, stability) in enumerate(zip(etas, convergence_stabilities)):
    ax3.annotate(f'{stability:.3f}', (eta, stability), textcoords="offset points",
                xytext=(0,10), ha='center', fontsize=10)

plt.tight_layout()
plt.show()

# In bảng tổng kết
print("\n" + "="*100)
print("PHÂN TÍCH KẾT QUẢ FEDADAM THEO THAM SỐ η")
print("="*100)

summary_df = pd.DataFrame(analysis_data)
summary_df.columns = ['η (Learning Rate)', 'Final Val Acc (Avg 10)', 'Best Val Acc', 'Convergence Stability (Std)']
print(summary_df.to_string(index=False, float_format='%.4f'))

print(f"\n🎯 ETA TỐT NHẤT (dựa trên Final Validation Accuracy - trung bình 10 round cuối): η = {best_eta}")
best_data = next(data for data in analysis_data if data['eta'] == best_eta)
print(f"   Final Validation Accuracy (Avg 10): {best_data['final_val_acc']:.4f}")
print(f"   Best Validation Accuracy: {best_data['best_val_acc']:.4f}")
print(f"   Convergence Stability: {best_data['convergence_stability']:.4f}")



In [ ]:
# Tạo biểu đồ bổ sung: Training và Validation curves cho tất cả eta
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Training và Validation Curves cho FedAdam với các η khác nhau', fontsize=16, fontweight='bold')

# Tạo colors dictionary động dựa trên số lượng eta
eta_list = sorted(results.keys())
colors_list = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
colors = {eta: colors_list[i % len(colors_list)] for i, eta in enumerate(eta_list)}

# Training Loss
ax1 = axes[0, 0]
for eta in results.keys():
    metrics = results[eta]['metrics']
    rounds = [round_data['round'] for round_data in metrics]
    train_losses = [round_data['train_loss'] for round_data in metrics]
    ax1.plot(rounds, train_losses, label=f'η = {eta}', color=colors[eta], linewidth=2)

ax1.set_title('Training Loss')
ax1.set_xlabel('Round')
ax1.set_ylabel('Training Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Validation Loss
ax2 = axes[0, 1]
for eta in results.keys():
    metrics = results[eta]['metrics']
    rounds = [round_data['round'] for round_data in metrics]
    val_losses = [round_data['val_loss'] for round_data in metrics]
    ax2.plot(rounds, val_losses, label=f'η = {eta}', color=colors[eta], linewidth=2)

ax2.set_title('Validation Loss')
ax2.set_xlabel('Round')
ax2.set_ylabel('Validation Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Training Accuracy
ax3 = axes[1, 0]
for eta in results.keys():
    metrics = results[eta]['metrics']
    rounds = [round_data['round'] for round_data in metrics]
    train_accs = [round_data['train_acc'] for round_data in metrics]
    ax3.plot(rounds, train_accs, label=f'η = {eta}', color=colors[eta], linewidth=2)

ax3.set_title('Training Accuracy')
ax3.set_xlabel('Round')
ax3.set_ylabel('Training Accuracy')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Validation Accuracy
ax4 = axes[1, 1]
for eta in results.keys():
    metrics = results[eta]['metrics']
    rounds = [round_data['round'] for round_data in metrics]
    val_accs = [round_data['val_acc'] for round_data in metrics]
    ax4.plot(rounds, val_accs, label=f'η = {eta}', color=colors[eta], linewidth=2)

ax4.set_title('Validation Accuracy')
ax4.set_xlabel('Round')
ax4.set_ylabel('Validation Accuracy')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
